<a href="https://colab.research.google.com/github/Vandan1009/python_1/blob/main/pspyk10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Create local file with the following data

Complete the following code using pyspark RDDs

movie.csv
movieId,moviename,director,year
200,shawshank,FrankD,1994
201,saw,JamesW,2004
202,avengers,JossW,2012
204,mi,ChristopherMcq,2018
205,wrongturn,DeclanO,2011

award.csv
catalogId,movieId,awards
11,200,2
12,200,3
13,201,1
14,202,3
15,202,5
16,204,4
17,205,1

In [1]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=868aeaa6a25a60753fc298e787f2899358f5493fb12ff4640795e80323977d4c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws,col,lit,sum,count

In [3]:
#Import libraries & Create SparkSession
spark=SparkSession.builder.appName("movies").getOrCreate()

In [4]:
spark

In [12]:
#Create two dataframes using respective csv files from local storage
df1=spark.createDataFrame([
[200,'shawshank','FrankD',1994],
[201,'saw','JamesW',2004],
[202,'avengers','JossW',2012],
[204,'mi','ChristopherMcq',2018],
[205,'wrongturn','DeclanO',2011]], schema=['movieId','moviename','director','year'])



In [13]:
df1.show()

+-------+---------+--------------+----+
|movieId|moviename|      director|year|
+-------+---------+--------------+----+
|    200|shawshank|        FrankD|1994|
|    201|      saw|        JamesW|2004|
|    202| avengers|         JossW|2012|
|    204|       mi|ChristopherMcq|2018|
|    205|wrongturn|       DeclanO|2011|
+-------+---------+--------------+----+



In [14]:
#Create a new column called "movieInfo" in movies dataframe which includes movieName & Director
df1.withColumn('moveInfo',concat_ws(' ','moviename','director')).show()

+-------+---------+--------------+----+-----------------+
|movieId|moviename|      director|year|         moveInfo|
+-------+---------+--------------+----+-----------------+
|    200|shawshank|        FrankD|1994| shawshank FrankD|
|    201|      saw|        JamesW|2004|       saw JamesW|
|    202| avengers|         JossW|2012|   avengers JossW|
|    204|       mi|ChristopherMcq|2018|mi ChristopherMcq|
|    205|wrongturn|       DeclanO|2011|wrongturn DeclanO|
+-------+---------+--------------+----+-----------------+



In [15]:
df2=spark.createDataFrame([[11,200,2],
[12,200,3],
[13,201,1],
[14,202,3],
[15,202,5],
[16,204,4],
[17,205,1]],schema=['catalogId','movieId','awards'])

In [16]:
df2.show()

+---------+-------+------+
|catalogId|movieId|awards|
+---------+-------+------+
|       11|    200|     2|
|       12|    200|     3|
|       13|    201|     1|
|       14|    202|     3|
|       15|    202|     5|
|       16|    204|     4|
|       17|    205|     1|
+---------+-------+------+



In [21]:
#Calculate total number of awards for each movie & display - movieID, movienName, awardsCount

# select c.moviename,c.movieId,sum(b.awards) as awardsCount
# from df2
# inner join df1 c
# on b.moveid==c.movieid
# group by moviename

# data=df2.join(df1,"movieId","inner")
# data.groupBy('moviename','movieId').sum('awards').show()

# data=df2.join(df1,"movieId","inner")
# data.groupBy('moviename','movieId').agg({'awards':'sum'}).show()

g=df2.join(df1,'movieId','inner')
g.groupBy('moviename','movieId').agg(sum('awards')).show()


+---------+-------+-----------+
|moviename|movieId|sum(awards)|
+---------+-------+-----------+
|shawshank|    200|          5|
|      saw|    201|          1|
| avengers|    202|          8|
|       mi|    204|          4|
|wrongturn|    205|          1|
+---------+-------+-----------+



In [27]:
#Get top three movies on the basis of number of awards
from pyspark.sql.functions import col
# data=df2.join(df1,"movieId","inner")
# data.groupBy('moviename').agg({'awards':'sum'}).sort(col('sum(awards)').desc()).limit(3).show()

data=df2.join(df1,'movieId','inner')
data.groupBy('moviename').agg(sum('awards')).sort('sum(awards)',ascending=False).limit(3).show()



+---------+-----------+
|moviename|sum(awards)|
+---------+-----------+
| avengers|          8|
|shawshank|          5|
|       mi|          4|
+---------+-----------+



In [28]:
#Get top three latest movies
from pyspark.sql.functions import col
#df1.sort(col('year').desc()).limit(3).show()

df1.sort('year',ascending=False).limit(3).show()

+-------+---------+--------------+----+
|movieId|moviename|      director|year|
+-------+---------+--------------+----+
|    204|       mi|ChristopherMcq|2018|
|    202| avengers|         JossW|2012|
|    205|wrongturn|       DeclanO|2011|
+-------+---------+--------------+----+

